# Week 2C (Advanced)

## Question 1

Suppose we perform the PCY algorithm to find frequent pairs, with market-basket data meeting the following specifications:

<ol>
<li>s, the support threshold, is 10,000.
<li>There are one million items, which are represented by the integers 0,1,...,999999.
<li>There are 250,000 frequent items, that is, items that occur 10,000 times or more.
<li>There are one million pairs that occur 10,000 times or more.
<li>There are P pairs that occur exactly once and consist of 2 frequent items.
<li>No other pairs occur at all.
<li>Integers are always represented by 4 bytes.
<li>When we hash pairs, they distribute among buckets randomly, but as evenly as possible; i.e., you may assume that each bucket gets exactly its fair share of the P pairs that occur once.
</ol>

Suppose there are S bytes of main memory. In order to run the PCY algorithm successfully, the number of buckets must be sufficiently large that most buckets are not large. In addition, on the second pass, there must be enough room to count all the candidate pairs. As a function of S, what is the largest value of P for which we can successfully run the PCY algorithm on this data? Demonstrate that you have the correct formula by indicating which of the following is a value for S and a value for P that is approximately (i.e., to within 10%) the largest possible value of P for that S.

<ol>
<li>S = 1,000,000,000; P = 20,000,000,000
<li>S = 100,000,000; P = 540,000,000
<li>S = 1,000,000,000; P = 35,000,000,000
<li>S = 1,000,000,000; P = 10,000,000,000
</ol>

In [55]:
"""
HINTS: 

The number of infrequent pairs per bucket on the first pass will be about P divided by the number of buckets.

A pair can only be a candidate pair for the second pass if it is in a frequent bucket. For the values of P and S found in this question, that can only occur if the bucket contains one of the 1,000,000 frequent pairs.

You must use a hash table to count candidate pairs on the second pass of PCY. This hash table takes 12 bytes per candidate pair.
bucket = 2 bytes for tresholds less then 2^16 four byte integers are replaced by bits 1:32 compression

Support treshold: 10,000 - One million pairs pass this treshold
Perfect hash function: Each bucket has m number of pairs hashed to it

1-pass:
- 1 million items * 4 bytes

2-pass:
Second pass n*m candidate pairs * 4 bytes

average number of infrequent pairs that will hash to the frequent buckets
- 12-byte triples for these pairs

1M freq buckets with 4P/S infreq items per buckets, needs 12*(1M)* 4P/S memory for triplets.

First Pass:
- Let there be S/4 buckets possible
- The infrequent pair for P will only be a candidate pair in Second pass if it is hashed to one of the buckets that contain one of the 1,000,000 frequent pairs. This means there is a probability of 1000000/(S/4) of in frequent pair being hashed to one of the frequent baskets.
==> so a total of  P∗((1,000,000)/(S/4))  pairs will be selected as a candidate pair.

Second Pass:
- each candidate pair takes 12 bytes, hence the max. pair possible will be 12 * candidate pairs = S
- using the number of candidate pairs we obtained from Pass 1, we get 12∗P∗((1,000,000)/(S/4))=S

===> doing some algebraic work we get P=S2/(4.8∗107)

"""

# Limits
LOWER, UPPER = 0.9, 1.1

values = [
    {"S": 1000000000, "P": 20000000000},
    {"S": 100000000, "P": 540000000},
    {"S": 1000000000, "P": 35000000000},
    {"S": 1000000000, "P": 10000000000}]

def p_max(S):
    # Amount space needed to find the number of infrequent pairs
    hash_table, bitmap = 0, 0
    S -= (1000000*4 + hash_table + bitmap + 250000*12)
    P = (S**2)/((4.8*10)**7) 
    return P
    
for v in values:
    p = p_max(v["S"])
    print(p)
    l, u = p * LOWER, p * UPPER
    if l <= v["P"] <= u:
        print(v)

1679615.3513982953
14732.526653588064
1679615.3513982953
1679615.3513982953


## Question 2

During a run of Toivonen's Algorithm with set of items {A,B,C,D,E,F,G,H} a sample is found to have the following maximal frequent itemsets: {A,B}, {A,C}, {A,D}, {B,C}, {E}, {F}. Compute the negative border. Then, identify in the list below the set that is NOT in the negative border.

<ol>
<li>{B,F}
<li>{B,D}
<li>{D}
<li>{B,E}
</ol>

In [7]:
"""
Having constructed the collection of frequent itemsets for the sample, we
next construct the negative border. This is the collection of itemsets that are not
frequent in the sample, but all of their immediate subsets (subsets constructed
by deleting exactly one item) are frequent in the sample.
"""

def extend_frequent(sets):
    "A subset of a maximal frequent itemset, such as the empty set, must itself be frequent"
    for f in sets:
        if len(f) > 1:
            for v in f:
                c = f.copy()
                c.remove(v)
                sets.append(c)
    return sets

# Set of items
items = set(["A", "B", "C", "D", "E", "F", "G", "H"])

# Maximal frequent itemsets
frequent_itemset = [set(["A", "B"]), set(["A", "C"]), set(["A", "D"]), set(["B", "C"]), set(["E"]), set(["F"])]

# Sets to test
#test = [set(["B", "F"]),set(["B", "D"]), set(["D"]), set(["B", "E"])]
#test = [set(["B", "E"]),set(["G", "H"]), set(["G"]), set(["C", "D"])]
test = [set(["D", "F"]),set(["B", "F"]), set(["D"]), set(["B", "D"])]
                
frequent_itemset = extend_frequent(frequent_itemset)
    
# Not frequent itemsets
negative_border = []

# Frequent itemsets
frequent = []

for t in test:
    count, freq = 0, False
    # Generate all possible subsets
    for v in t:
        c = t.copy()
        c.remove(v)
        for f in frequent_itemset:
            # Count all subsets found in the frequent itemset
            if c == f:
                count += 1
            # If test set if a subset of a frequent set -> frequent
            if t.issubset(f):
                freq = True
        # If all subsets are found in the maximal frequent itemsets
        if count == len(t):
            negative_border.append(t)
        if freq:
            frequent.append(t)
        
print(frequent)
print(negative_border)

[{'D'}]
[{'D', 'F'}, {'D', 'B'}]
